In [ ]:
#import dependencies
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from scipy import stats
from config import spotify_api_key
import seaborn as sns 
import random

In [ ]:
# What factors influence the success of a song 

# Presentation 
# Cleaning - Bryan talks about b/c took out null values and added columns
# quantified success with ranking columns combined views/streams and overall rank

# Bryan - youtube vs spotify 

# Coble - Danceability coorelation to overall success, other dancabilit coor

# Paulin - streams and views by album 

# Taylor - success single vs an album, top 100 singles and top 100 albums, if likes or comments have a factor

# Kevals - top youtube videos official vs. unofficial and where falls in overall rank 

# API stuff

#left to do: 

# 4 visuals
# slides - frame for the slides 

#Next Steps - Bryan is going to create skeleton powerpoint

In [ ]:
data = "./resources/Spotify_Youtube.csv"
spot_yt_df = pd.read_csv(data)

In [ ]:
#clean DF for analysis and add new columns
spot_yt_df["Views"] = spot_yt_df["Views"].fillna(0)
spot_yt_df["Stream"] = spot_yt_df["Stream"].fillna(0)
spot_yt_df["Danceability"] = spot_yt_df["Danceability"].fillna(0)


#remove duplicate tracks
spot_yt_df = spot_yt_df.drop_duplicates(subset= "Track", keep= "first")

# Add "Views" and "Stream" together and create a new column "Combined Data"
spot_yt_df['Combined Views and Streams'] = (spot_yt_df['Stream'] + spot_yt_df['Views'])


spot_yt_df['YouTube Rank'] = spot_yt_df['Views'].rank(ascending=False, method='min')
spot_yt_df['Spotify Rank'] = spot_yt_df['Stream'].rank(ascending=False, method='min')
spot_yt_df['Overall Rank'] = spot_yt_df['Combined Views and Streams'].rank(ascending=False, method='min')



spot_yt_df

In [ ]:
#YT views vs spotify streams - Bryan
yt_vs_spotify = spot_yt_df[["Artist", "Track", "Album", "Views", "Stream", "Combined Views and Streams", "YouTube Rank", "Spotify Rank", "Overall Rank"]]

# DF by combined data in descending order
yt_vs_spotify = yt_vs_spotify.sort_values(by="Combined Views and Streams", ascending=False).reset_index(drop=True)

# Convert DF to string type
yt_vs_spotify = yt_vs_spotify.astype(str)

yt_vs_spotify 

In [ ]:
# DF for youtube views
yt_df = spot_yt_df.sort_values(by="Views", ascending=False).head(20)

# DF for spotify streams
spotify_df = spot_yt_df.sort_values(by="Stream", ascending=False).head(20)

# DF for combined views and streams
combined_df = spot_yt_df.sort_values(by="Combined Views and Streams", ascending=False).head(20)

yt_df[["Artist", "Track", "Spotify Rank"]]

In [ ]:
spotify_df[["Artist", "Track", "YouTube Rank"]]

In [ ]:
combined_df[["Artist", "Track", "YouTube Rank", "Spotify Rank"]]

In [ ]:
def vs_plot(x_values, y_values, ylabel, x,y):
    
    slope, intercept, rvalue, pvalue, stderr = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope, 2)) + "x +" + str(round(intercept, 2))
    plt.scatter(x_values, y_values)
    plt.plot(x_values, regress_values, "r-")
    plt.title("Correlation of Views to Streams")
    plt.xlabel('Stream')
    plt.ylabel('Views')
    plt.annotate(line_eq, (x,y), fontsize=15,color="red")

    print(f"The r-value is: {rvalue**2}")
    
    plt.show()

In [ ]:
# Linear regression on Views to Streams
x_values = spot_yt_df["Views"]
y_values = spot_yt_df["Stream"]
vs_plot(x_values, y_values, "Stream", 30000000, 300000)

In [ ]:
#15 most danceable and streamed on YT and Spot
#15 most overall popular songs on spot and YT
#data frame for danceability against popularity - Coble
dance_pop_df = spot_yt_df[["Artist", "Track", "Album", "Danceability", "Views", "Stream", "Tempo", "Energy", "Combined Views and Streams", "Overall Rank"]]

dance_pop_df = dance_pop_df.sort_values(by=['Danceability'], ascending=[False])

dance_pop_df.reset_index(drop=True).head(50)

# Want to graph
# Danceability doesn't necessarily correlate with overall rank
# Frequency of top artists for danceability

In [ ]:
# Function to format numbers in billions
def billions_formatter(x, pos):
    return f'{x / 1e9:.1f}B'  # Format numbers in billions with one decimal place

# Assuming you have a DataFrame named dance_pop_df with 'Danceability' and 'Combined Views and Streams' columns

# Create a scatter plot
plt.figure(figsize=(10, 6))
scatterplot = sns.scatterplot(data=dance_pop_df, x='Danceability', y='Combined Views and Streams', alpha=0.5)

# Customize y-axis labels to use the custom billions_formatter
y_formatter = FuncFormatter(billions_formatter)
scatterplot.yaxis.set_major_formatter(y_formatter)

plt.title('Scatter Plot: Danceability vs. Success')
plt.xlabel('Danceability')
plt.ylabel('Combined Views and Streams (Billions)')

plt.grid(True)
plt.show()

In [ ]:
# Fit a linear regression model
slope, intercept, r_value, p_value, std_err = stats.linregress(dance_pop_df['Overall Rank'], dance_pop_df['Danceability'])

# Create a scatter plot with the regression line
plt.figure(figsize=(10, 6))
plt.scatter(dance_pop_df['Tempo'], dance_pop_df['Danceability'], marker='o', alpha=0.5)
plt.plot(dance_pop_df['Tempo'], intercept + slope * dance_pop_df['Tempo'], color='red', linestyle='-', label=f'Linear Fit (R-squared = {r_value**2:.2f})')
plt.title('Scatter Plot with Linear Fit: Tempo vs. Danceability')
plt.xlabel('Tempo (BPM)')
plt.ylabel('Danceability')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Assuming you have the DataFrame dance_pop_df with columns 'Energy', 'Danceability', 'Artist', 'Stream', and 'Views'

# Filter the DataFrame to select rows where Danceability is between 0.6 and 1
filtered_df = dance_pop_df[(dance_pop_df['Danceability'] >= 0.6) & (dance_pop_df['Danceability'] <= 1)]

# Count the frequency of each artist
artist_counts = filtered_df['Artist'].value_counts().reset_index()
artist_counts.columns = ['Artist', 'Frequency']

# Group by the 'Artist' column and sum the 'Stream' and 'Views' columns
grouped_df = filtered_df.groupby('Artist').agg({'Stream': 'sum', 'Views': 'sum', 'Energy': 'mean'}).reset_index()

# Exclude artists that appear only once
grouped_df = grouped_df[grouped_df['Stream'] > 1]

# Calculate the 'Success' column as the sum of 'Stream' and 'Views'
grouped_df['Success'] = grouped_df['Stream'] + grouped_df['Views']

# Merge the 'Frequency' column back into the grouped DataFrame
result_df = grouped_df.merge(artist_counts, on='Artist', how='left')

# Rename the columns for clarity
result_df.columns = ['Artist', 'Combined Stream', 'Combined Views', 'Energy', 'Success', 'Frequency']

# Sort the result DataFrame by 'Frequency' in descending order
result_df = result_df.sort_values(by='Frequency', ascending=False)

# Add a 'Dance Artist Rank' column based on the 'Success' column
result_df['Dance Artist Rank'] = result_df['Success'].rank(ascending=False, method='dense').astype(int)

# Sort the result DataFrame by 'Dance Artist Rank' in ascending order
result_df = result_df.sort_values(by='Frequency', ascending=False)

# Reset the index after sorting
result_df = result_df.reset_index(drop=True).astype(str)

# Print the sorted and indexed DataFrame
result_df

In [ ]:
# Function to format numbers in billions
def billions_formatter(x, pos):
    return f'{x / 1e9:.1f}B'  # Format numbers in billions with one decimal place

# Assuming you have a DataFrame named result_df with 'Energy' and 'Success' columns

# Randomly select a subset of data points (e.g., 1000 data points)
subset_size = 1850
subset_df = result_df.sample(n=subset_size, random_state=42)

# Create a scatterplot for 'Energy' vs. 'Success'
plt.figure(figsize=(10, 6))
scatterplot = sns.scatterplot(data=subset_df, x='Energy', y='Success', alpha=0.3, marker='o', edgecolors='k')

# Customize y-axis labels to use the custom billions_formatter
scatterplot.yaxis.set_major_formatter(FuncFormatter(billions_formatter))

# Set plot labels and title
plt.title('Scatter Plot: Energy vs. Success')
plt.xlabel('Energy (0-1)')
plt.ylabel('Success (Billions)')

plt.grid(True)
plt.show()


In [ ]:
#album with the most streams on spotify & most views on Youtube - Paulin

#Separate Album dataframe from cleaned df
comb_streams_views = spot_yt_df[["Artist","Album", "Album_type","Combined Views and Streams"]]

#Take out singles
new_comb_streams_views = comb_streams_views[comb_streams_views.Album_type=="album"]
new_comb_streams_views

#group by Album & sum Combined Views & Streams
grouped_df = new_comb_streams_views.groupby('Album')["Combined Views and Streams"].sum().reset_index()
grouped_df = grouped_df.sort_values("Combined Views and Streams", ascending=False)
grouped_df

#Get the top ten
Top_ten_df = grouped_df.head(10)
Top_ten_df = Top_ten_df.reset_index()
Top_ten_df


#Graph top ten
plt.bar(Top_ten_df["Album"], Top_ten_df["Combined Views and Streams"], color='r', align="center")
plt.xlabel("Album")
plt.ylabel("Combined Views and Streams (Billions)") 
plt.title("Top 10 Albums")
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Original Spotify Youtube Data Frame (Taylor)
spot_yt_df[["Album_type","Artist", "Track","Combined Views and Streams", "Overall Rank",]]
spot_yt_df

In [ ]:
#Create a single_album_df to separate it from the cleaned spot_yt_df (Taylor)
single_album_df = spot_yt_df[["Artist", "Track", "Album_type", "Stream", "Views", "Comments", "Likes"]]
single_album_df = single_album_df.sort_values("Stream", ascending=False).astype(str)
single_album_df.head(10)

In [ ]:
#Added two columns to determine success, cleaned up the data a bit further, sorted values (Taylor)
single_album_df = spot_yt_df[["Artist", "Track", "Album_type", "Views", "Stream",
                              "Combined Views and Streams", "Overall Rank"]]
single_album_df = single_album_df.sort_values("Combined Views and Streams", ascending=False).astype(str).reset_index(drop=True)
single_album_df.rename(columns={"Album_type": "Track_Origin"}, inplace=True)
single_album_df.head(20)

In [ ]:
#Filtering out to see Top 100, it collapsed the data. (Taylor)

single_album_df = spot_yt_df[["Album_type", "Combined Views and Streams", "Overall Rank"]]
single_album_df = single_album_df.sort_values("Combined Views and Streams", ascending=False).astype(str).reset_index(drop=True)
single_album_df.rename(columns={"Album_type": "Track_Origin"}, inplace=True)
single_album_df.head(100)

In [ ]:
# Bar Chart created to visualize single/album and which one is more successful (Taylor)
# Analysis Single (Blue) VS Album (Green)
# Blue represents single's success in the overall ranked top 100 (All single's success added)
# Green represents album's succecss in the overall ranked top 100 (All album's success added)
# Album is more successful by about more than double. Give or take 30 billion more combined views and streams.
# This data shows the overall top 100 consisting of both single/album. 
# Challenge - Does not account for individual data shows more of a group, which would require further cleaning and plotting.
data = {
    "Track_Origin": ["Single", "Album", "Single", "Album"],
    "Combined Views and Streams": [1500000000, 3500000000, 20000000000, 50000000000]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Define a custom function to format y-axis labels in billions
def billions_formatter(x, pos):
    return f'{x / 1e9:.0f}B'  # Format numbers in billions

# Create a bar chart
plt.figure(figsize=(8, 6))
bars = plt.bar(df["Track_Origin"], df["Combined Views and Streams"])
plt.xlabel("Track_Origin")
plt.ylabel("Combined Views and Streams (in Billions)")
plt.title("Top 100 Single/Album and Success")

# Use the custom billions_formatter function for y-axis labels
y_formatter = FuncFormatter(billions_formatter)
plt.gca().yaxis.set_major_formatter(y_formatter)

# Set the color of the "Album" bar to green
bars[3].set_color("green")  # Index 1 corresponds to the "Album" bar

plt.show()

In [ ]:
#offical video success against non official videos - compared to spotify track - Keval
df = spot_yt_df[["Artist", "Track", "Stream", "Views", "official_video", "Combined Views and Streams"]]

official_video_df = df.rename(columns={'Stream': 'Spotify Stream','Views':'Youtube Views','official_video':'Official Video'})

official_sorted_stream = official_video_df.sort_values(by = "Youtube Views", ascending = False).reset_index(drop=True)

top_100_streamed_songs = official_sorted_stream.head(100)

# Count the occurrences of 'Official Video' values
true_false_count = top_100_streamed_songs["Official Video"].value_counts().reset_index()


# Create a bar plot with two bars for 'Official Songs' and 'Non-Official Songs'
bars = plt.bar(true_false_count["index"], true_false_count["Official Video"], color=['blue', 'green'], \
               width=0.8, align='center')

# Set the legend labels for each bar
bars[0].set_label("Official Video")
bars[1].set_label("Unofficial Video")

# Create the legend
plt.legend()

# Modify the tick labels to match the legend labels
tick_location = [value for value in true_false_count["index"]]
plt.xticks(tick_location, ["Official Video", "Unofficial Video"])

plt.title("Official and Unofficial Video in Top 100 Streamed Songs")
plt.ylabel("Top 100 Streamed Songs Count")
plt.xlabel("Official or Unofficial Video")

plt.show()

In [ ]:
#Licensed song success against unLicensed video - compared to spotify track - Anyone
licensed_song = spot_yt_df[["Artist", "Track", "Stream", "Views", "Licensed","Combined Views and Streams"]]

licensed_song_df = licensed_song.rename(columns={'Stream': 'Spotify Stream','Views':'Youtube Views'})
licensed_sorted_stream = licensed_song_df.sort_values(by = "Combined Views and Streams", ascending = False).reset_index(drop=True)

top_100_licensed_unlicensed_songs = licensed_sorted_stream.head(100)

true_false_count_licensed = top_100_licensed_unlicensed_songs["Licensed"].value_counts().reset_index()
bars = plt.bar(true_false_count_licensed["index"], true_false_count_licensed["Licensed"], color=['blue', 'green'], width=0.8, align='center')

# Set the legend labels for each bar
bars[0].set_label("Licensed Song")
bars[1].set_label("Unlicensed Song")

# Create the legend
plt.legend()

# Modify the tick labels to match the legend labels
tick_location = [value for value in true_false_count_licensed["index"]]
plt.xticks(tick_location, ["Licensed Song", "Unlicensed Song"])

plt.title("Licensed and Unlicensed Song in Top 100 Streamed Songs")
plt.ylabel("Top 100 Streamed Songs Count")
plt.xlabel("Licensed or Unlicensed Video")

plt.show()

In [ ]:
#API: Genre against popularity for Spotify - Keval

top_100_songs_for_API = spot_yt_df[["Artist", "Track","Url_spotify","Album", "Stream", "Spotify Rank"]]\
.sort_values(by="Stream", ascending=False).reset_index(drop=True).head(100)


get_artist_id = top_100_songs_for_API["Url_spotify"].str.split("/")

artist_df = pd.DataFrame({"artist_data": get_artist_id})
artist_id= []
for row in artist_df["artist_data"]:
#     if row[4] not in artist_id:
    artist_id.append(row[4])
        
artist_id_df = pd.DataFrame({"Artist": top_100_songs_for_API["Artist"],"Artist ID":artist_id})
followers = []
# Replace 'YOUR_ACCESS_TOKEN' with your actual Spotify access token

# Spotify API endpoint for the artist


# Set the headers with the Authorization token
headers = {
    "Authorization": f"Bearer {spotify_api_key}"
}

for index, row in artist_id_df.iterrows():
    url = "https://api.spotify.com/v1/artists/{}".format(row[1])
    

# Make the GET request
    response = requests.get(url, headers=headers).json()
    
    followers.append(response["followers"]["total"])
    
artist_followers_df = pd.DataFrame({"Artist": top_100_songs_for_API["Artist"],"Artist ID":artist_id,"Followers":followers})
artist_followers_df

In [ ]:

merged_df = pd.merge(top_100_songs_for_API, artist_followers_df, on='Artist',how='left').drop_duplicates().reset_index(drop=True)


unique_merged_df = merged_df.drop_duplicates().reset_index(drop=True)

most_followed_artists = unique_merged_df[["Artist","Followers"]].sort_values(by="Followers", ascending=False)
unique_most_followed_artists = most_followed_artists.drop_duplicates().reset_index(drop=True)

pd.options.display.float_format = '{:.0f}'.format
sorted_unique_artist_followers = unique_most_followed_artists.sort_values(by="Followers", ascending=False).head(50)
# .plot(kind="bar",fontsize=25,figsize=(30,30))

def billions_formatter(x, pos):
    return f'{x / 1e6:.1f}M'  # Format numbers in billions with one decimal place

# Create a bar chart
plt.figure(figsize=(12, 8))  # Set the figure size (adjust as needed)
plt.bar(sorted_unique_artist_followers["Artist"], sorted_unique_artist_followers['Followers'])
plt.xlabel("Artist")
plt.ylabel("Followers")
plt.title("Top 50 Most Followed Artists")


# Use the custom billions_formatter function for y-axis labels
y_formatter = FuncFormatter(billions_formatter)
plt.gca().yaxis.set_major_formatter(y_formatter)

# Rotate x-axis labels for better readability (adjust rotation angle as needed)
plt.xticks(rotation=90)

plt.tight_layout()  # Adjust layout to prevent labels from being cut off
plt.show()
# grouped_df.sort_values(by="Stream", ascending=False)

In [ ]:
Top_listened_artists = unique_merged_df[["Artist","Stream", "Followers"]].sort_values(by="Stream", ascending=False)
groupby_artists = pd.DataFrame(Top_listened_artists.groupby(by="Artist")["Stream"].sum())
groupby_artists.sort_values(by="Stream", ascending=False).drop_duplicates().reset_index(drop=True).head(50)

pd.options.display.float_format = '{:.0f}'.format
groupby_50_artists = groupby_artists.sort_values(by="Stream", ascending=False).head(50)
# .plot(kind="bar",fontsize=25,figsize=(30,30))

def billions_formatter(x, pos):
    return f'{x / 1e9:.1f}B'  # Format numbers in billions with one decimal place

# Create a bar chart
plt.figure(figsize=(12, 8))  # Set the figure size (adjust as needed)
plt.bar(groupby_50_artists.index, groupby_50_artists['Stream'])
plt.xlabel("Artist")
plt.ylabel("Streams (in Billions)")
plt.title("Top 50 Listened Artists")


# Use the custom billions_formatter function for y-axis labels
y_formatter = FuncFormatter(billions_formatter)
plt.gca().yaxis.set_major_formatter(y_formatter)

# Rotate x-axis labels for better readability (adjust rotation angle as needed)
plt.xticks(rotation=90)

plt.tight_layout()  # Adjust layout to prevent labels from being cut off
plt.show()